In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
# import nlopt
import cadquery as cq
from geomdl import NURBS

In [ ]:

fillet_radius = 5.0
base_radius = 15.5 - 0.2
top_radius = 35.0
hole_radius = 6.5 + 0.1
height_to_hole_top = 20 + 3.0
height_to_hole = height_to_hole_top + hole_radius

# create the base cylinder
force_reciever = (
    cq.Workplane("XY")
    .circle(base_radius)
    .extrude(height_to_hole + hole_radius + 6)
)

# cut the screw hole
force_reciever = (
    force_reciever.transformed((0,-90,0))
    .moveTo(height_to_hole, 0)
    .circle(hole_radius)
    .extrude(base_radius*2, both=True, combine="cut")
)

# add recieving surface
force_reciever = (
    force_reciever.faces("<Z")
    .transformed((0,90,0))
    .circle(base_radius)
    .workplane(offset=-5)
    .circle(top_radius)
    .loft(combine=True)
    .faces(">>Z[-3]")
    .edges()
    .fillet(2.5)
    .faces("<Z")
    .extrude(-3)
    .faces("<<Z[-2]")
    .edges()
    .fillet(1.0)
)

force_reciever.export("force_reciever.stl")


In [ ]:
desired_diameter = 60.0
base_plate_height = 3.0
screw_hole_diameter = 25.0
screw_radius = 1.5
exploded_factor = 1.5
no_screws = 6
thickness = 1.0
squeeze_tolerance = 0.2
model_radius = 15.0

base1 = (
    cq.Workplane("XZ")
    .circle(desired_diameter/2)
    # .polyline(base_radius_outline)
    # .close()
    .extrude(base_plate_height, both=True)
    # Create the seal clamp
    .faces("<Y")
    .workplane()
    .circle((desired_diameter - screw_hole_diameter)/2 + screw_radius*2)
    .extrude(-(thickness * 2 - squeeze_tolerance), combine='cut')
    # Cut screw holes
    .faces(">Y")
    .polarArray(radius=screw_hole_diameter, startAngle=0, angle=360, count=no_screws, rotate=True)
    .circle(screw_radius*1.15)
    .cutThruAll()
    # Hollow the plate
    .faces(">Y")
    .workplane()
    .circle(model_radius)
    .extrude(-base_plate_height*2, combine='cut')
    .translate((0, thickness*3*exploded_factor, 0))
)


faces_parallel_to_z = base1.faces("|X")
sorted_face = faces_parallel_to_z.faces 

base.export("base1.stl")

In [14]:
r1 = (
    cq.Workplane()
    .box(10, 25, 1)
    .faces(">Z")
    .rarray(1, 2, 1, 10)
    .slot2D(8, 1, 0)
    .cutThruAll()
)

r1.export("r1.stl")

In [ ]:
b = cq.Workplane().circle(40).extrude(4).faces('>Z').workplane()

# e = b.polarArray(3.2,0,360,8,rotate=False).slot2D(3,1,90).cutThruAll()

e = b.polarArray(3.2,0,360,8,rotate=True).slot2D(3,1,90).cutThruAll()

e.export("e.stl")

In [29]:
base = (
    cq.Workplane("XZ")
    .circle(50)
    .extrude(4)
    # Screw holes
    .faces('>Y')
    .workplane()
)

base2 = base.rotate((0, 0, 0), (0, 1, 0), 45).polarArray(radius=30, startAngle=0, angle=360, count=4, rotate=True).slot2D(10,2,0).cutThruAll()


base2.export("base2.stl")

In [3]:
workplane0 = cq.Workplane("XY").rect(10,2).workplane()

rectangle_set = [(11,2),(12,2),(12,3),(12.5,3),(12,2.5),(11,2.5),(10,2.5),(11,2.5),(10,2)]

result = None
for i in range(0,9):
    workplane1 = workplane0.transformed(offset=cq.Vector(0, 0, 0.5),rotate=cq.Vector(10, 0, 0)).rect(*rectangle_set[i]).workplane()
    if result == None:
        result = workplane1.loft(combine=True)
    else:
        nextpart = workplane1.loft(combine=True)
        result = result + nextpart
    workplane0 = workplane0.transformed(offset=cq.Vector(0, 0, 0.5),rotate=cq.Vector(10, 0, 0)).rect(*rectangle_set[i]).workplane()

# result.export("test_loft.stl")


In [2]:
# rectangular frame points and quick CadQuery build (cell 5)

# parameters (change as needed)
width = 60.0
height = 40.0
wall = 4.0        # frame wall thickness
thickness_z = 5.0 # extrusion height

# basic validation
if wall * 2 >= min(width, height):
    raise ValueError("wall too large for given width/height")

# outer rectangle points (clockwise)
outer_pts = [
    (-width / 2, -height / 2),
    ( width / 2, -height / 2),
    ( width / 2,  height / 2),
    (-width / 2,  height / 2),
]

# inner rectangle points (clockwise)
inner_w = width - 2 * wall
inner_h = height - 2 * wall
inner_pts = [
    (-inner_w / 2, -inner_h / 2),
    ( inner_w / 2, -inner_h / 2),
    ( inner_w / 2,  inner_h / 2),
    (-inner_w / 2,  inner_h / 2),
]

# combine points so inner loop is reversed (creates a hole when extruded)
profile_pts = outer_pts + inner_pts[::-1]

# profile_pts is the set of points that define a rectangular frame
# you can use it directly with CadQuery's polyline to create the frame:
rect_frame = cq.Workplane("XY").polyline(profile_pts).close().extrude(thickness_z)

# export for inspection
rect_frame.export("rectangular_frame.stl")

In [10]:
# handle increments and angular_section
increments = 12

# create array to store workplanes
workplanes = []

# define initial workplane [cross_section0]
initial_pts = profile_pts

# calculate minimum offset for each increment
def calculate_minimum_x_offset(inc_angle_deg, x_length, centre_offset):
    inc_angle_rad = math.radians(inc_angle_deg)
    # x_min_offset = ((x_length + centre_offset) / 2) * (1 - math.cos(inc_angle_rad))
    x_min_offset = (centre_offset) * (1 - math.cos(inc_angle_rad))
    return x_min_offset

# calculate minimum offset for each increment
def calculate_minimum_z_offset(inc_angle_deg, x_length, centre_offset):
    inc_angle_rad = math.radians(inc_angle_deg)
    z_min_offset = ((x_length + centre_offset) / 2) * math.sin(inc_angle_rad)
    # z_min_offset = (centre_offset) * math.sin(inc_angle_rad)
    return z_min_offset

# create initial workplane
workplane0 = (cq.Workplane("XY")
            .polyline(initial_pts)
            .close()
            .faces()
            .workplane()
)

# append initial workplane to the list
workplanes.append(workplane0)

# calculate the increment angle
angle = 360 / increments  # degrees

x_prev = max(p[0] for p in initial_pts) - min(p[0] for p in initial_pts)

profile = None
for i in range(0,(increments)):

    # (1) DERIVE THE X AND Z OFFSETS

    # extract curve points
    curve_points = initial_pts

    # Get the current rectangle dimensions
    x_len = max(p[0] for p in initial_pts) - min(p[0] for p in initial_pts)
    # y_len = helpers.find_max_y(curve_points) - helpers.find_min_y(curve_points)
    # print(f"Increment {i}: x_length = {x}, y_length = {y}")

    # Get the current workplane
    workplane_now = workplanes[i].faces().workplane()

    # Calculate the z and x offsets
    # x_variance = calculate_x_inc_variance(x_len, x_prev)

    z_offset = calculate_minimum_z_offset(angle, x_len, centre_offset=0)
    x_offset = calculate_minimum_x_offset(angle, x_len, centre_offset=0)

    # (2) CREATE THE WORKPLANES AND LOFT

    workplane_new = workplane_now.transformed(offset=cq.Vector(x_offset, 0, z_offset),rotate=cq.Vector(0, angle, 0)).polyline(curve_points).close().faces().workplane()
    workplanes.append(workplane_new)

    # print(f"completed increment {i+1}")

# Get the maximum y value for the cap
last_curve_points = initial_pts

profile = workplanes[-1].loft(ruled=True,combine=True)

profile.export("circular_lofted_frame.stl")

In [9]:
# increments per 90 degree section of the full angular domain
increments = 9

# angular period/section
angular_section = 90

# create counter to track and store angular progression
total_angle = 0

workplanes = []

# workplane0 = cq.Workplane("XY").rect(30,15).workplane()
result0 = cq.Workplane("XY").rect(30,15).faces().workplane()
workplanes.append(result0)

baseline_set = [(30, 15), (30, 15), (30, 15), (30, 15), (30, 15), (30, 15), (30, 15), (30, 15), (30, 15),
                (30, 15), (30, 15), (30, 15), (30, 15), (30, 15), (30, 15), (30, 15), (30, 15), (30, 15),
                (30, 15), (30, 15), (30, 15), (30, 15), (30, 15), (30, 15), (30, 15), (30, 15), (30, 15),
                (30, 15), (30, 15), (30, 15), (30, 15), (30, 15), (30, 15), (30, 15), (30, 15), (30, 15),
                (30, 15), (30, 15), (30, 15), (30, 15), (30, 15), (30, 15), (30, 15), (30, 15), (30, 15)]

rectangle_set = [(30,15),(30,14),(32,16),(31,15.5),(29,15),(28,14),(27,14.5),(29,14),(30,15),
                 (28,15),(30,14),(32,16),(31,15.5),(29,15),(28,14),(27,14.5),(29,14),(30,15),
                 (28,15),(30,14),(32,16),(31,15.5),(29,15),(28,14),(27,14.5),(29,14),(30,15),
                 (28,15),(30,14),(32,16),(31,15.5),(29,15),(28,14),(27,14.5),(29,14),(30,15)]

# calculate minimum offset for each increment
def calculate_minimum_z_offset(inc_angle_deg, x_length, centre_offset):
    inc_angle_rad = math.radians(inc_angle_deg)
    z_min_offset = ((x_length + centre_offset) / 2) * math.sin(inc_angle_rad)
    return z_min_offset

# calculate minimum offset for each increment
def calculate_minimum_x_offset(inc_angle_deg, x_length, centre_offset):
    inc_angle_rad = math.radians(inc_angle_deg)
    x_min_offset = ((x_length + centre_offset) / 2) * (1 - math.cos(inc_angle_rad))
    return x_min_offset

# calculate the angle for each increment 
def calculate_angle(increments):
    angle = angular_section / increments
    return angle

def calculate_x_inc_variance(x_length, x_length_prev):
    x_variance = x_length - x_length_prev
    return x_variance

# dataset = baseline_set
dataset = rectangle_set

result = None
for i in range(0,(increments)):

    # (1) DERIVE THE X AND Z OFFSETS

    # Get the current rectangle dimensions
    x = dataset[i][0]
    # y = dataset[i][1]
    # calculate the increment angle
    angle = calculate_angle(increments)

    # update angle counter
    total_angle += angle

    # Calculate the z offset
    z_offset = calculate_minimum_z_offset(angle, x, centre_offset=1)

    # calculate the x offset
    x_offset = calculate_minimum_x_offset(angle, x, centre_offset=1)
    
    # print(f"Increment {i}: x_offset = {x_offset}, z_offset = {z_offset}")

    x_variance = calculate_x_inc_variance(x, x_prev) if i > 0 else 0

    # (2) CREATE THE WORKPLANES AND LOFT

    result0 = workplanes[i].faces().workplane()
    # result1 = result0.faces().workplane().transformed(offset=cq.Vector(x_offset-(x_variance/2), 0, z_offset),rotate=cq.Vector(0, angle, 0)).rect(*dataset[i]).faces().workplane()
    # if result == None:
    #     result = result1.faces().workplane().loft(combine=True)
    # else:
    #     nextpart = result1.faces().workplane().loft(combine=True)
    #     result = result + nextpart
    # result0 = result1
    result1 = result0.faces().workplane().transformed(offset=cq.Vector(x_offset-(x_variance/2), 0, z_offset),rotate=cq.Vector(0, angle, 0)).rect(*dataset[i]).faces().workplane()
    # workplane0 = workplane0.transformed(offset=cq.Vector(x_offset-(x_variance/2), 0,z_offset),rotate=cq.Vector(0, angle, 0)).rect(*dataset[i]).workplane()

    # workplane_new = workplane_now.transformed(offset=cq.Vector(x_offset-(x_variance/2), 0, z_offset),rotate=cq.Vector(0, angle, 0)).polyline(curve_points).close().faces().workplane()
    workplanes.append(result1)

    x_prev = x

result = workplanes[increments].loft(ruled=True,combine=True)

result.export("test_loft.stl")

In [ ]:
# increments per 90 degree section of the full angular domain
increments = 9

# angular period/section
angular_section = 90

# create counter to track and store angular progression
total_angle = 0

# create array to store workplanes
workplanes = []

workplane0 = cq.Workplane("XY").rect(30,15).workplane()
# result0 = cq.Workplane("XY").rect(30,15)
workplanes.append(workplane0)

baseline_set = [(30, 15), (30, 15), (30, 15), (30, 15), (30, 15), (30, 15), (30, 15), (30, 15), (30, 15),
                (30, 15), (30, 15), (30, 15), (30, 15), (30, 15), (30, 15), (30, 15), (30, 15), (30, 15),
                (30, 15), (30, 15), (30, 15), (30, 15), (30, 15), (30, 15), (30, 15), (30, 15), (30, 15),
                (30, 15), (30, 15), (30, 15), (30, 15), (30, 15), (30, 15), (30, 15), (30, 15), (30, 15),
                (30, 15), (30, 15), (30, 15), (30, 15), (30, 15), (30, 15), (30, 15), (30, 15), (30, 15)]

rectangle_set = [(30,15),(30,14),(32,16),(31,15.5),(29,15),(28,14),(27,14.5),(29,14),(30,15),
                 (28,15),(30,14),(32,16),(31,15.5),(29,15),(28,14),(27,14.5),(29,14),(30,15),
                 (28,15),(30,14),(32,16),(31,15.5),(29,15),(28,14),(27,14.5),(29,14),(30,15),
                 (28,15),(30,14),(32,16),(31,15.5),(29,15),(28,14),(27,14.5),(29,14),(30,15)]

# calculate minimum offset for each increment
def calculate_minimum_z_offset(inc_angle_deg, x_length, centre_offset):
    inc_angle_rad = math.radians(inc_angle_deg)
    z_min_offset = ((x_length + centre_offset) / 2) * math.sin(inc_angle_rad)
    return z_min_offset

# calculate minimum offset for each increment
def calculate_minimum_x_offset(inc_angle_deg, x_length, centre_offset):
    inc_angle_rad = math.radians(inc_angle_deg)
    x_min_offset = ((x_length + centre_offset) / 2) * (1 - math.cos(inc_angle_rad))
    return x_min_offset

# calculate the angle for each increment 
def calculate_angle(increments):
    angle = angular_section / increments
    return angle

def calculate_x_inc_variance(x_length, x_length_prev):
    x_variance = x_length - x_length_prev
    return x_variance

dataset = baseline_set
# dataset = rectangle_set

result = None
for i in range(0,(increments)):

    # (1) DERIVE THE X AND Z OFFSETS

    workplane_now = workplanes[i].faces().workplane()

    # Get the current rectangle dimensions
    x = dataset[i][0]
    # y = dataset[i][1]
    # calculate the increment angle
    angle = calculate_angle(increments)

    # update angle counter
    total_angle += angle

    # Calculate the z offset
    z_offset = calculate_minimum_z_offset(angle, x, centre_offset=1)

    # calculate the x offset
    x_offset = calculate_minimum_x_offset(angle, x, centre_offset=1)
    
    # print(f"Increment {i}: x_offset = {x_offset}, z_offset = {z_offset}")

    x_variance = calculate_x_inc_variance(x, x_prev) if i > 0 else 0

    # (2) CREATE THE WORKPLANES AND LOFT

    workplane_new = workplane_now.transformed(offset=cq.Vector(x_offset-(x_variance/2), 0, z_offset),rotate=cq.Vector(0, angle, 0)).rect(*dataset[i]).workplane()
    # if result == None:
        # result = workplane_new.loft(combine=True)
    # else:
        # nextpart = workplane_new.loft(combine=True)
        # result = result + nextpart
    # workplane0 = workplane0.transformed(offset=cq.Vector(x_offset-(x_variance/2), 0,z_offset),rotate=cq.Vector(0, angle, 0)).rect(*dataset[i]).workplane()

    workplanes.append(workplane_new)

    # result1 = result0.faces().workplane().transformed(offset=cq.Vector(x_offset-(x_variance/2), 0, z_offset),rotate=cq.Vector(0, angle, 0)).rect(*dataset[i])
    # if result == None:
    #     result = result1.faces().workplane().loft(combine=True)
    # else:
    #     nextpart = result1.faces().workplane().loft(combine=True)
    #     result = result + nextpart
    # # result0 = result1
    # result0 = result0.faces().workplane().transformed(offset=cq.Vector(x_offset-(x_variance/2), 0, z_offset),rotate=cq.Vector(0, angle, 0)).rect(*dataset[i])
    # workplane0 = workplane0.transformed(offset=cq.Vector(x_offset-(x_variance/2), 0,z_offset),rotate=cq.Vector(0, angle, 0)).rect(*dataset[i]).workplane()

    x_prev = x

print(f"no. of workplanes: {len(workplanes)}")

result = workplanes[increments].loft(combine=True)

result.export("test_loft3.stl")

no. of workplanes: 10


: 

In [ ]:
workplane0 = cq.Workplane("XY").rect(30,15).extrude(0.5, both=True).workplane()

workplane1 = workplane0.transformed(offset=cq.Vector(0.34, 0, 2.72),rotate=cq.Vector(0, 10, 0)).rect(30,15).extrude(0.5, both=True)
# result = workplane0 + workplane1

workplane2 = workplane1.transformed(offset=cq.Vector(0.34, 0, 2.72),rotate=cq.Vector(0, 10, 0)).rect(30,15).extrude(0.5, both=True)

result = workplane0 + workplane1 + workplane2

# result = workplane1.loft(combine=True)
result.export("test_loft2.stl")

: 

: 

: 

: 